In [8]:
import numpy as np
import pandas as pd
import multiprocessing
import time
from sklearn.linear_model import LogisticRegression as lr
from sklearn.model_selection import train_test_split
from dask import compute, delayed
import dask.multiprocessing
import warnings
warnings.filterwarnings("ignore")
num_core = multiprocessing.cpu_count()
print('Number of available cores is', num_cores)

Number of available cores is 4


In [6]:
auto = pd.read_csv("data/Auto.csv", na_values="?").dropna()
auto["mpg_high"] = np.where(auto.mpg > auto.mpg.median(), 1, 0)
auto["orgn1"] = auto.origin == 1
auto["orgn2"] = auto.origin == 2
X = auto[["cylinders", "displacement", "horsepower", "weight", "acceleration", "year", "orgn1", "orgn2"]]
y = auto["mpg_high"]
st = time.time()
mse_1 = []

for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.65, random_state=i)
    clf = lr(solver="lbfgs", max_iter=200, n_jobs=1).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    m = ((y_test - y_pred) ** 2).mean()
    mse_1.append(m)
    
avg_mse_1 = np.array(mse_1).mean()

print("Average error rate : {}".format(avg_mse_1))
print("Computation Time: {} ".format(time.time() - st))

Average error rate : 0.10072463768115941
Computation Time: 3.013232707977295 


In [10]:
def cal_err_rate(boot_num, seed, data_X, data_y):
    X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, train_size=0.65, random_state=seed)
    clf = lr(solver="lbfgs", max_iter=200, n_jobs=1).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    mse = ((y_test - y_pred) ** 2).mean()
    return mse
st = time.time()
mse2 = []
for i in range(100):
    mse2.append(delayed(cal_err_rate)(i, i, X, y))
results = compute(mse2, schedule=dask.multiprocessing.get, num_workers=num_core)

print("Average error rate : {}".format(np.mean(results)))
print("Computation time : {} ".format(time.time() - st))

Average error rate : 0.10072463768115941
Computation time : 4.433130979537964 
